# Import all necessary libraries

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import random
# Dependencies and Setup
# SQL Alchemy
from sqlalchemy import create_engine

In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, MetaData, PrimaryKeyConstraint

from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
import sqlite3

# Load and clean data set
## EV charging station by location

In [3]:
stations_df = pd.read_csv('../project-3/data/alt_fuel_stations.csv')
stations_df.head()

/Users/kaipham/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,EV Pricing (French),LPG Nozzle Types,Hydrogen Pressures,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Restricted Access
0,ELEC,LADWP - Truesdale Center,11797 Truesdale St,NaN,Sun Valley,CA,91352,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ELEC,LADWP - West LA District Office,1394 S Sepulveda Blvd,NaN,Los Angeles,CA,90024,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ELEC,Los Angeles Convention Center,1201 S Figueroa St,West hall and South hall,Los Angeles,CA,90015,NaN,213-741-1151,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ELEC,LADWP - John Ferraro Building,111 N Hope St,Across Hope,Los Angeles,CA,90012,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ELEC,LADWP - Haynes Power Plant,6801 E 2nd St,NaN,Long Beach,CA,90803,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
stations_df.columns

Index(['Fuel Type Code', 'Station Name', 'Street Address',
       'Intersection Directions', 'City', 'State', 'ZIP', 'Plus4',
       'Station Phone', 'Status Code', 'Expected Date',
       'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
       'BD Blends', 'NG Fill Type Code', 'NG PSI', 'EV Level1 EVSE Num',
       'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Other Info', 'EV Network',
       'EV Network Web', 'Geocode Status', 'Latitude', 'Longitude',
       'Date Last Confirmed', 'ID', 'Updated At', 'Owner Type Code',
       'Federal Agency ID', 'Federal Agency Name', 'Open Date',
       'Hydrogen Status Link', 'NG Vehicle Class', 'LPG Primary',
       'E85 Blender Pump', 'EV Connector Types', 'Country',
       'Intersection Directions (French)', 'Access Days Time (French)',
       'BD Blends (French)', 'Groups With Access Code (French)',
       'Hydrogen Is Retail', 'Access Code', 'Access Detail Code',
       'Federal Agency Code', 'Facility Type', 'CNG Dispenser N

In [5]:
# keep only wanted columns & Reorder - make ID column first
stations_df = stations_df[['ID','Station Name','Street Address','City','State','ZIP','Station Phone','Status Code',
                           'Groups With Access Code','Access Days Time','Geocode Status',
       'Latitude', 'Longitude','Open Date','EV Connector Types','Access Code', 'Facility Type', 'EV Pricing']]

In [6]:
# Rename column names
stations_df.rename(columns={'ID': 'id', 'Station Name':'station_name', 'Street Address': 'street_address',
                            'City': 'city', 'State': 'state','ZIP': 'zip','Station Phone': 'station_phone',
                            'Status Code': 'status_code', 'Groups With Access Code': 'groups_access_code', 
                            'Access Days Time': 'access_days_time', 'Geocode Status': 'geocode status',
                            'Latitude': 'latitude', 'Longitude': 'longitude','Open Date':'open_date','EV Connector Types':'ev_connector_types',
                            'Access Code':'access_code', 'Facility Type':'facility_type' , 'EV Pricing':'ev_pricing'
                            }, inplace=True)

In [7]:
stations_df.head()

,id,station_name,street_address,city,state,zip,station_phone,status_code,groups_access_code,access_days_time,geocode status,latitude,longitude,open_date,ev_connector_types,access_code,facility_type,ev_pricing
0,1517,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,91352,NaN,E,Private,Fleet use only,GPS,34.248319,-118.387971,1999-10-15,CHADEMO J1772 J1772COMBO,private,UTILITY,NaN
1,1519,LADWP - West LA District Office,1394 S Sepulveda Blvd,Los Angeles,CA,90024,NaN,E,Private,NaN,200-8,34.052542,-118.448504,2020-02-28,J1772,private,UTILITY,Free
2,1523,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,213-741-1151,E,Public,24 hours daily; pay lot,GPS,34.040539,-118.271387,1995-08-30,J1772,public,PARKING_GARAGE,NaN
3,1525,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,NaN,E,Private,For fleet and employee use only,GPS,34.059133,-118.248589,1999-10-15,CHADEMO J1772 J1772COMBO,private,UTILITY,NaN
4,1531,LADWP - Haynes Power Plant,6801 E 2nd St,Long Beach,CA,90803,NaN,E,Private,Fleet use only,GPS,33.759802,-118.096665,2018-05-01,CHADEMO J1772 J1772COMBO,private,UTILITY,NaN


In [8]:
print(stations_df.columns)

Index(['id', 'station_name', 'street_address', 'city', 'state', 'zip',
       'station_phone', 'status_code', 'groups_access_code',
       'access_days_time', 'geocode status', 'latitude', 'longitude',
       'open_date', 'ev_connector_types', 'access_code', 'facility_type',
       'ev_pricing'],
      dtype='object')


In [9]:
stations_df.fillna(0)

,id,station_name,street_address,city,state,zip,station_phone,status_code,groups_access_code,access_days_time,geocode status,latitude,longitude,open_date,ev_connector_types,access_code,facility_type,ev_pricing
0,1517,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,91352,0,E,Private,Fleet use only,GPS,34.248319,-118.387971,1999-10-15,CHADEMO J1772 J1772COMBO,private,UTILITY,0
1,1519,LADWP - West LA District Office,1394 S Sepulveda Blvd,Los Angeles,CA,90024,0,E,Private,0,200-8,34.052542,-118.448504,2020-02-28,J1772,private,UTILITY,Free
2,1523,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,213-741-1151,E,Public,24 hours daily; pay lot,GPS,34.040539,-118.271387,1995-08-30,J1772,public,PARKING_GARAGE,0
3,1525,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,0,E,Private,For fleet and employee use only,GPS,34.059133,-118.248589,1999-10-15,CHADEMO J1772 J1772COMBO,private,UTILITY,0
4,1531,LADWP - Haynes Power Plant,6801 E 2nd St,Long Beach,CA,90803,0,E,Private,Fleet use only,GPS,33.759802,-118.096665,2018-05-01,CHADEMO J1772 J1772COMBO,private,UTILITY,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50265,204568,MARIPOSAON3RD SPACE 13,269 S Mariposa Ave,Los Angeles,CA,90004,888-758-4389,E,Public,24 hours daily,GPS,34.069280,-118.299479,2022-01-10,J1772,public,0,0
50266,204569,MARIPOSAON3RD SPACE 14,269 S Mariposa Ave,Los Angeles,CA,90004,888-758-4389,E,Public,24 hours daily,GPS,34.069280,-118.299538,2022-01-10,J1772,public,0,0
50267,204570,MARIPOSAON3RD SPACE 11,269 S Mariposa Ave,Los Angeles,CA,90004,888-758-4389,E,Public,24 hours daily,GPS,34.069204,-118.299361,2022-01-10,J1772,public,0,0
50268,204571,TOWN OF NATICK KMS #2,165 Mill St,Natick,MA,1760,888-758-4389,E,Public,24 hours daily,GPS,42.291105,-71.384538,2022-01-10,J1772,public,0,0


In [10]:
stations_df.size

904860

In [11]:
stations_df.describe()

,id,latitude,longitude
count,50270.000000,50270.000000,50270.000000
mean,150032.847344,37.661903,-98.485662
std,44779.024067,5.012525,19.776861
min,1517.000000,18.334138,-159.788556
25%,115940.250000,34.023393,-118.333923
50%,166442.500000,37.927566,-95.337205
75%,182445.750000,41.122613,-80.142582
max,204572.000000,64.852466,0.000000


## use web scrape to see how average temperatures in different regions have increased over time

In [12]:
url = 'https://science2017.globalchange.gov/chapter/6/'
table = pd.read_html(url)
temp_df = table[2]
temp_df

,NCA Region,Change in Annual Average Temperature,Change in Annual Average Maximum Temperature,Change in Annual Average Minimum Temperature
0,Contiguous U.S.,1.23°F,1.06°F,1.41°F
1,Northeast,1.43°F,1.16°F,1.70°F
2,Southeast,0.46°F,0.16°F,0.76°F
3,Midwest,1.26°F,0.77°F,1.75°F
4,Great Plains North,1.69°F,1.66°F,1.72°F
5,Great Plains South,0.76°F,0.56°F,0.96°F
6,Southwest,1.61°F,1.61°F,1.61°F
7,Northwest,1.54°F,1.52°F,1.56°F
8,Alaska,1.67°F,1.43°F,1.91°F
9,Hawaii,1.26°F,1.01°F,1.49°F


In [13]:
## convert tabulated data to html if needed

temp_table = temp_df.to_html()

In [19]:
# Store filepath in a variable
#station_file = "Resources/DataOne.csv"
carbon_dioxide_file = '../Project-3/data/US_and_world_co2_emission.csv'

In [23]:
carbon_dioxide_df = pd.read_csv(carbon_dioxide_file)

In [25]:
carbon_dioxide_df.rename(columns={'ï»¿ENTITY':'ENTITY'},inplace=True)

In [28]:
carbon_dioxide_df.columns

Index(['Entity', 'Code', 'Year', 'Annual CO2 emissions (tons )'], dtype='object')

In [26]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
carbon_dioxide_df = pd.read_csv(carbon_dioxide_file, encoding="ISO-8859-1")

In [29]:
## load states CO2 emission

US_co2 = carbon_dioxide_df.loc[carbon_dioxide_df["Entity"] == "United States"]
US_co2_df = US_co2.tail(30)
US_co2_df

,Entity,Code,Year,Annual CO2 emissions (tons )
188,United States,USA,1988,4.890000e+09
189,United States,USA,1989,4.950000e+09
190,United States,USA,1990,5.120000e+09
191,United States,USA,1991,5.070000e+09
192,United States,USA,1992,5.170000e+09
193,United States,USA,1993,5.280000e+09
194,United States,USA,1994,5.380000e+09
195,United States,USA,1995,5.440000e+09
196,United States,USA,1996,5.630000e+09
197,United States,USA,1997,5.700000e+09


In [ ]:
US_co2_df.to_csv("..data/US_co2_df")

In [30]:
states_co2 = "../data/StatesCO2_emission.csv"
##states_co2 = pd.read_csv(states_co2, error_bad_lines=False)
states_co2

'../data/StatesCO2_emission.csv'

In [31]:
engine = create_engine("sqlite:///data/db.sqlite")

meta = MetaData()
meta.drop_all(engine)